In [ ]:
import sys, os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.regularizers import l2
import matplotlib.pyplot as plt

#loading previously saved features and labels
a = np.load('./features.npy')
B = np.load('./labels.npy')
#taking the mean of the pixel features
a -= np.mean(A, axis = 0)
#taking the normal standard deviation of the mean of the pixel features
a /= np.std(A, axis = 0)

#Sklearn's test train split function using pseudo random numbers so that everytime the code is run, the output of the split is the same. 80/20 split train/test
A_train, A_test, B_train, B_test = train_test_split(a, B, test_size = 0.1, random_state = 5)
#making a seperate split within the train split, to give a validation set
A_train, A_valid, B_train, B_valid = train_test_split(A_train, B_train, test_size = 0.1, random_state = 5)

#saving test split using a numpy array to be used later
np.save('ftestAngry', A_test)
np.save('ltestAngry', B_test)

#cnn model design
model = Sequential()

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = (48, 48, 1), data_format = 'channels_last', kernel_regularizer = l2(0.01)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation = 'sigmoid'))

model.summary()

#Compliling the model with adam optimizer and binary crossentropy loss
model.compile(loss = 'binary_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

model_info = model.fit(np.array(A_train), np.array(B_train),
          batch_size = 64,
          epochs = 100,
          verbose = 1,
          validation_data = (np.array(A_valid), np.array(B_valid)),
          callbacks = [EarlyStopping(monitor = 'val_loss', patience = int(5))],
          shuffle = True)
#saving the  model to be used later for testing
CNN_json = model.to_json()
with open("CNNAngry.json", "w") as json_file:
    json_file.write(CNN_json)
    model.save_weights("CNNAngry.h5")
    #adding an element to the end of 'A', then converting the string into a float with a 32-bit single-precision floating-point format
    A.append(face_features.astype('float32'))
#puts features into an array
A = np.asarray(A)
#adding another dimension
A = np.expand_dims(A, -1)
#retrieving labels for training
B = pd.get_dummies(data['emotion']).as_matrix()

#storing features and labels, respectively, using numpy
np.save('features', A)
np.save('labels', B)

print("Features: "+str(len(A[0])))
print("Dataset Images:"+str(len(A)))